In [1]:
from ultralytics import YOLO
from ultralytics.data.converter import convert_coco

from sklearn.model_selection import train_test_split

import pandas as pd
import re
import yaml

from PIL import Image
import os
import json

In [2]:
root_dir = 'C:\\Users\\natha\\Documents\\PythonNotebooks\\yolo\\coco_converted\\'
image_dir = 'C:\\Users\\natha\\Documents\\PythonNotebooks\\yolo\\files\\image\\images\\gallery\\800\\'
image_dir_2 = 'C:\\Users\\natha\\Documents\\PythonNotebooks\\yolo\\files\\pill_pictures\\'

In [3]:
with open('files/instances_default.json') as file:
    labels = json.load(file)
with open('files/instances_default_two.json') as file:
    labels_2 = json.load(file)

In [4]:
for row in labels_2['images']:
    row['id'] = row['id'] + 10**6
for row in labels_2['annotations']:
    row['image_id'] = row['image_id'] + 10**6

In [5]:
annotations = labels['annotations'] + labels_2['annotations']
images = labels['images'] + labels_2['images']
image_ids = {item['image_id'] for item in annotations}

x_train, x_test = train_test_split(list(image_ids), train_size=0.85, random_state=42)

train_annotations = []
test_annotations = []

train_images = []
test_images = []

for annotation in annotations:
    if annotation['image_id'] in x_train:
        train_annotations.append(annotation)
    else:
        test_annotations.append(annotation)

for image in images:
    if image['id'] in x_train:
        train_images.append(image)
    elif image['id'] in x_test:
        test_images.append(image)

In [6]:
labels['annotations'] = train_annotations
labels['images'] = train_images

with open('files/annotations/train.json', 'w') as file:
    json.dump(labels, file)

labels['annotations'] = test_annotations
labels['images'] = test_images

with open('files/annotations/val.json', 'w') as file:
    json.dump(labels, file)


In [7]:
convert_coco(labels_dir='files/annotations')

Annotations C:\Users\natha\Documents\PythonNotebooks\yolo\files\annotations\train.json: 100%|██████████| 697/697 [00:00
Annotations C:\Users\natha\Documents\PythonNotebooks\yolo\files\annotations\val.json: 100%|██████████| 124/124 [00:00<0

COCO data converted successfully.
Results saved to C:\Users\natha\Documents\PythonNotebooks\yolo\coco_converted


In [8]:
os.makedirs('coco_converted/images/train/', exist_ok=True)
os.makedirs('coco_converted/images/val/', exist_ok=True)
for image in train_images:
    filename = image['file_name']
    if os.path.isfile(f'{image_dir}{filename}'):
        input_path = f'{image_dir}{filename}'
    else:
        input_path = f'{image_dir_2}{filename}'
    output_path = f'coco_converted/images/train/{filename}'

    with Image.open(input_path) as img:
        # Resize the image to 640x640
        img_resized = img.resize((640, 640))
    
        # Save the resized image to the output path
        img_resized.save(output_path)

for image in test_images:
    filename = image['file_name']
    if os.path.isfile(f'{image_dir}{filename}'):
        input_path = f'{image_dir}{filename}'
    else:
        input_path = f'{image_dir_2}{filename}'
    output_path = f'coco_converted/images/val/{filename}'

    with Image.open(input_path) as img:
        # Resize the image to 640x640
        img_resized = img.resize((640, 640))
    
        # Save the resized image to the output path
        img_resized.save(output_path)

In [9]:
{item['id'] - 1: item['name'] for item in labels['categories']}

{0: 'Pill',
 1: 'Round',
 2: 'Capsule',
 3: 'Rectangular',
 4: 'Blue',
 5: 'White',
 6: 'Yellow',
 7: 'Red',
 8: 'Green',
 9: 'Pink',
 10: 'Ellipse'}

In [10]:
def make_yaml_file():
    data = {
        'path': root_dir,  # dataset root dir
        'train': 'images/train',  # train images (relative to 'path') 4 images
        'val': 'images/val',  # val images (relative to 'path') 4 images
        'names': {item['id'] - 1: item['name'] for item in labels['categories']}
    }

    with open('files/pill_data.yaml', 'w') as file:
        yaml.dump(data, file, default_flow_style=False)
make_yaml_file()

In [11]:
model = YOLO("yolo11n.pt")
model.train(data='files/pill_data.yaml', epochs=300)

Ultralytics 8.3.47  Python-3.11.8 torch-2.2.1 CUDA:0 (NVIDIA GeForce GTX 1080, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=files/pill_data.yaml, epochs=300, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxe

train: Scanning C:\Users\natha\Documents\PythonNotebooks\yolo\coco_converted\labels\train... 697 images, 0 backgrounds,


train: New cache created: C:\Users\natha\Documents\PythonNotebooks\yolo\coco_converted\labels\train.cache


val: Scanning C:\Users\natha\Documents\PythonNotebooks\yolo\coco_converted\labels\val... 124 images, 0 backgrounds, 0 c

val: New cache created: C:\Users\natha\Documents\PythonNotebooks\yolo\coco_converted\labels\val.cache


...

    151/300      4.45G      0.255      0.287     0.8679        112        640: 100%|██████████| 44/44 [00:13<00:00,  3.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<0

                   all        124       1170      0.857      0.859      0.913      0.849



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    152/300      3.29G     0.2617     0.2947     0.8571        279        640:   9%|▉         | 4/44 [00:01<00:15,  2.5


KeyboardInterrupt: 

In [ ]:
model.save('300_epoch_n.pt')

In [ ]:
test_images = os.listdir('files/test_pictures')
test_images = [os.path.join('files/test_pictures', filename) for filename in test_images if filename[0] != '.']
test_images = [Image.open(filepath).resize((640, 640)) for filepath in test_images]
results = model(test_images)